In [1]:
!pip install mlflow boto3 awscli optuna imbalanced-learn

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.1.3 requires docutils<0.22,>=0.20, but you have docutils 0.16 which is incompatible.


In [ ]:
!aws configure

In [3]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-100-26-36-125.compute-1.amazonaws.com:5000/")

In [4]:
# Set or create an experiment
mlflow.set_experiment("E ML Algos with HP Tuning")

<Experiment: artifact_location='s3://om-mlflow-bucket/622937322449081937', creation_time=1741368134336, experiment_id='622937322449081937', last_update_time=1741368134336, lifecycle_stage='active', name='E ML Algos with HP Tuning', tags={}>

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna


In [6]:
df = pd.read_csv('/content/reddit_processed.csv').dropna()


In [8]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Multinomial Naive Bayes

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

X = df['clean_comment']
y = df['category']

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 10000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Multinomial Naive Bayes
def objective_mnb(trial):
    alpha = trial.suggest_float('alpha', 1e-4, 1.0, log=True)  # Tuning the smoothing parameter

    # MultinomialNB model setup
    model = MultinomialNB(alpha=alpha)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Multinomial Naive Bayes, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_mnb, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = MultinomialNB(alpha=best_params['alpha'])

    # Log the best model with MLflow, passing the algo_name as "MultinomialNB"
    log_mlflow("MultinomialNB", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Multinomial Naive Bayes
run_optuna_experiment()


[I 2025-03-07 17:41:42,864] A new study created in memory with name: no-name-2ddd1425-146a-4051-acb6-5d888ab90a31
[I 2025-03-07 17:41:42,881] Trial 0 finished with value: 0.7096686213009682 and parameters: {'alpha': 0.030007282256800907}. Best is trial 0 with value: 0.7096686213009682.
[I 2025-03-07 17:41:42,897] Trial 1 finished with value: 0.7089867721260057 and parameters: {'alpha': 0.022652508370870544}. Best is trial 0 with value: 0.7096686213009682.
[I 2025-03-07 17:41:42,913] Trial 2 finished with value: 0.7078958134460658 and parameters: {'alpha': 0.000936357406166838}. Best is trial 0 with value: 0.7096686213009682.
[I 2025-03-07 17:41:42,929] Trial 3 finished with value: 0.7080321832810582 and parameters: {'alpha': 0.008323462466573778}. Best is trial 0 with value: 0.7096686213009682.
[I 2025-03-07 17:41:42,944] Trial 4 finished with value: 0.7083049229510432 and parameters: {'alpha': 0.1461365037183057}. Best is trial 0 with value: 0.7096686213009682.
[I 2025-03-07 17:41:42,

🏃 View run MultinomialNB_SMOTE_TFIDF_Trigrams at: http://ec2-100-26-36-125.compute-1.amazonaws.com:5000/#/experiments/622937322449081937/runs/3d03e5ee7fff4494a785fa27876e38d6
🧪 View experiment at: http://ec2-100-26-36-125.compute-1.amazonaws.com:5000/#/experiments/622937322449081937
